# **Dependencies**

In [ ]:
! pip install torch transformers peft accelerate bitsandhbytes datsets

In [1]:
from huggingface_hub import login
login(token="hf_peeXvbkzEPXLEhbuygburSUrJdoMQvUmldPygfygvfeb")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Siddharth\.cache\huggingface\token
Login successful


In [5]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer , BitsAndBytesConfig , Seq2SeqTrainer , Seq2SeqTrainingArguments , DataCollatorForSeq2Seq
from peft import get_peft_model , LoraConfig , prepare_model_for_kbit_training , TaskType
from datasets import load_dataset , Dataset
import torch
from accelerate import Accelerator
import pandas as pd

# **Data Processing**

In [3]:
model_id = "google/flan-t5-base"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bnb_config,
    device_map = "auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)
model.config.use_cache=False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

In [6]:
dataset_id = "ShenRuililin/MedicalQnA"
initial_dataset = load_dataset(dataset_id)
initial_dataset = initial_dataset["train"]
print(initial_dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 16412
})


In [7]:
df = pd.DataFrame(initial_dataset)
df.dropna(inplace=True)
df.to_csv("train.csv",index=False)
load_df = pd.read_csv("train.csv")
dataset = Dataset.from_pandas(load_df)

In [8]:
prefix = "Please answer this question: "
suffix = " Answer : "
def preprocess_function(examples):
   inputs = [prefix + doc + suffix for doc in examples["input"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
   labels = tokenizer(text_target=examples["output"], 
                      max_length=512,         
                      truncation=True)
   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [9]:
# Map the preprocessing function across our dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

# **Training**

In [10]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.01,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=["q","v"]
)
model = get_peft_model(model,lora_config)

In [11]:
training_arg = Seq2SeqTrainingArguments(
    output_dir = "./results",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate = 2e-4,
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=200,
    num_train_epochs=1,
    push_to_hub=False
)
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    args=training_arg
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddharthmagesh007 (velammal-edu-in). Use `wandb login --relogin` to force relogin


  0%|          | 0/4102 [00:00<?, ?it/s]

KeyboardInterrupt: 

# **Saving the Model**

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model_id = "google/flan-t5-base"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True
)
fine_tuned_path = r"flan-t5-base-MedicalQnA\results/checkpoint-5000/"
peft_model= PeftModel.from_pretrained(model,fine_tuned_path,from_transformers=True,device_map="auto")

model = peft_model.merge_and_unload()
model.push_to_hub("siddharth-magesh/flan-t5-base-MedicalQnA")

# **Inference**

In [ ]:
#inference on GPU 
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "siddharth-magesh/Tiny_Lllama-sqlQuries-fine_tuned"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto",trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Move the model to GPU
# model.to('cuda')

prefix = "Please answer this question: "
suffix = " Answer : "

input = "what causes high blood pressure ? "
prompt =   prefix + input + suffix

# Encode the prompt
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate the output
output = model.generate(**inputs, max_new_tokens=512)

# Decode the output
text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated SQL query
print(text)